<a href="https://colab.research.google.com/github/nicchic/NLP/blob/main/tune_mental_gpa2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch rouge_score

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [ ]:
from datasets import load_dataset

ds = load_dataset("Amod/mental_health_counseling_conversations")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
print(ds['train'].column_names)


['Context', 'Response']


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

def prepare_inputs(examples):

    combined_texts = [context + " " + response for context, response in zip(examples['Context'], examples['Response'])]

    # Tokenize the combined text
    tokenized_outputs = tokenizer(combined_texts,
                                  padding="max_length",
                                  truncation=True,
                                  max_length=128)

    # The labels are the same as input_ids for language modeling tasks
    tokenized_outputs["labels"] = tokenized_outputs["input_ids"].copy()

    # Return the required columns
    return {
        "input_ids": tokenized_outputs["input_ids"],
        "attention_mask": tokenized_outputs["attention_mask"],
        "labels": tokenized_outputs["labels"]
    }

# Apply the tokenization function to the dataset
tokenized_ds = ds.map(prepare_inputs, batched=True, remove_columns=ds['train'].column_names)



Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3512
    })
})

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=4,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    remove_unused_columns=False,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train']
)

In [ ]:
trainer.train()


Step,Training Loss
500,2.230400
1000,2.224800
1500,1.965100
2000,1.821200
2500,1.723300
3000,1.635600
3500,1.584100


TrainOutput(global_step=3512, training_loss=1.882695642853652, metrics={'train_runtime': 629.9457, 'train_samples_per_second': 22.3, 'train_steps_per_second': 5.575, 'total_flos': 917657616384000.0, 'train_loss': 1.882695642853652, 'epoch': 4.0})

In [ ]:
attention_mask = torch.tensor(tokenized_ds['train']['attention_mask'][0]).unsqueeze(0).to(device)


In [ ]:
input_ids = torch.tensor(tokenized_ds['train']['attention_mask'][0]).unsqueeze(0).to(device)

output = model.generate(input_ids,
                        attention_mask=attention_mask,
                        pad_token_id=tokenizer.eos_token_id,
                        max_length= 200,
                        do_sample=True,
                        top_p=0.8,
                        )

print(tokenizer.decode(output[0], skip_special_tokens=True))


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""]

""if ( $IsBound!== true


In [ ]:
output = model.generate(input_ids,
                        max_length= 200,
                        attention_mask=attention_mask,
                        pad_token_id=tokenizer.eos_token_id,
                        num_beams=5,
                        do_sample= False,
                        no_repeat_ngram_size =4
                        )

print(tokenizer.decode(output[0]))

""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""]

I'm sorry to hear that you are having a difficult time with your boyfriend. It sounds like he has a lot on his mind at times, but it seems like he has no interest in what you are going through. I would suggest that you talk to him about this and see if he would be willing to talk to you about what you are


In [ ]:
# Greedy decoding
output = model.generate(input_ids,
                        attention_mask=attention_mask,
                        max_new_tokens=128,
                        do_sample=False,
                        pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(output[0], skip_special_tokens=True))

""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

This is a great question. I think it is great that you are reaching out to ask this question. I think it is important that you reach out to someone who is willing to help you. I would encourage you to reach out to anyone who is willing to help you. I would also encourage you to reach out to anyone who is willing to help you. I would also encourage you to reach out to anyone who is willing to help you. I would also encourage you


In [16]:
pip install rouge_score

In [17]:
pip install evaluate


In [18]:
from evaluate import load
rouge = load('rouge')

In [19]:
generated_text_top ="If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why everyone feels that way, since you can't change the person.Start by talking to them about whatever it is that they are feeling.  Then, for each reason they feel so strongly that they would like to go anywhere, you can talk them on a regular basis."
output_beam = "I'm sorry to hear that you are having a difficult time with your boyfriend. It sounds like he has a lot on his mind at times, but it seems like he has no interest in what you are going through. I would suggest that you talk to him about this and see if he would be willing to talk to you about what you are"

reference_text = "Here are few things that you could do to make a positive change of you feel about yourself. Take baby steps towards your goals but take these steps everyday. List all the things that you want to do in life and then start working on one thing at a time. Give yourself credit for even the smallest accomplishment and don't forget to celebrate the fact that you've tried. always believe in yourself, you are worth everything to your loved ones"


In [20]:
rouge_result_top_p = rouge.compute(predictions=[generated_text_top], references=[reference_text])

print("\nROUGE Scores (Top-p Sampling):")

def display_rouge_scores(rouge_scores):
    for rouge_type, score in rouge_scores.items():
        print(f"{rouge_type.upper()}: {score:.4f}")

display_rouge_scores(rouge_result_top_p)


ROUGE Scores (Top-p Sampling):
ROUGE1: 0.3218
ROUGE2: 0.0000
ROUGEL: 0.1264
ROUGELSUM: 0.1264


In [21]:
rouge_result_beam = rouge.compute(predictions=[output_beam], references=[reference_text])
print("\nROUGE Scores (Beam Search):")
display_rouge_scores(rouge_result_beam)


ROUGE Scores (Beam Search):
ROUGE1: 0.3108
ROUGE2: 0.0411
ROUGEL: 0.1757
ROUGELSUM: 0.1757
